In [40]:
import pandas as pd
import requests
from git import Repo, GitCommandError
import os
import time

# Commit and push the changes to the GitHub repository
def commit_and_push_changes(repo, files):
    repo.git.add(files)
    try:
        repo.index.commit('Update main.tex with product page description and add email announcement and press release')
        origin = repo.remote(name='origin')
        for attempt in range(3):  # Retry up to 3 times
            try:
                origin.pull(rebase=True)  # Pull changes from the remote to avoid conflicts
                origin.push()
                print("Changes pushed to GitHub repository successfully.")
                break
            except GitCommandError as e:
                print(f"Attempt {attempt + 1}: Failed to push changes to GitHub repository: {e}")
                time.sleep(5)  # Wait for 5 seconds before retrying
        else:
            print("Failed to push changes after multiple attempts.")
    except GitCommandError as e:
        print(f"Failed to commit changes to GitHub repository: {e}")

# Function to escape LaTeX special characters
def escape_latex(text):
    escape_chars = {
        '_': r'\_',
        '&': r'\&',
        '%': r'\%',
        '$': r'\$',
        '#': r'\#',
        '{': r'\{',
        '}': r'\}',
        '~': r'\textasciitilde{}',
        '^': r'\textasciicircum{}',
        '\\': r'\textbackslash{}'
    }
    return ''.join(escape_chars.get(c, c) for c in str(text))

# Function to split parameter names and descriptions
def split_parameters(params_desc):
    if pd.isna(params_desc) or not params_desc.strip():
        return ""
    params = params_desc.split('\n')
    split_params = [param.split(' - ') for param in params if ' - ' in param]
    formatted_params = "\n".join([f"\\textbf{{{escape_latex(name.strip())}}}: {escape_latex(desc.strip())}" for name, desc in split_params])
    return formatted_params

# Function to generate the product page description based on the form data
def generate_product_page_description(row):
    sections = []
    
    plugin_name = escape_latex(row.get("Name of Plugin", ""))
    main_description = escape_latex(row.get("Main description (15 - 25 words)", ""))
    category = escape_latex(row.get("Category", ""))
    compatibility = escape_latex(row.get("Compatibility", ""))
    pricing = escape_latex(row.get("Pricing", ""))
    keywords = [escape_latex(kw) for kw in row.get("Keywords (3 - 6)", "").split(', ')]
    params_desc = row.get("Parameter names and short descriptions", "")
    unique_features = escape_latex(row.get("What makes this plugin unique?", ""))
    installation_instructions = escape_latex(row.get("How to install", ""))
    usage_instructions = escape_latex(row.get("How do you use the plugin?", ""))
    common_use_cases = escape_latex(row.get("What are some common use cases for the plugin?", ""))
    additional_info = escape_latex(row.get("Is there any additional information or notes?", ""))
    
    # Split and format the parameters
    formatted_params = split_parameters(params_desc)
    
    if main_description:
        sections.append(f"\\subsection*{{Description}}\n{main_description}")
    
    if category:
        sections.append(f"\\subsection*{{Category}}\n{category}")
    
    if compatibility:
        sections.append(f"\\subsection*{{Compatibility}}\n{compatibility}")
    
    if formatted_params:
        sections.append(f"\\subsection*{{Features}}\n{formatted_params}")
    
    if unique_features:
        sections.append(f"\\subsection*{{What Makes This Plugin Unique?}}\n{unique_features}")
    
    if pricing:
        sections.append(f"\\subsection*{{Pricing}}\n{pricing}")
    
    if keywords:
        sections.append(f"\\subsection*{{Keywords}}\n{', '.join(keywords)}")
    
    if installation_instructions:
        sections.append(f"\\subsection*{{Installation Instructions}}\n{installation_instructions}")
    
    if usage_instructions:
        sections.append(f"\\subsection*{{Usage Instructions}}\n{usage_instructions}")
    
    if common_use_cases:
        sections.append(f"\\subsection*{{Common Use Cases}}\n{common_use_cases}")
    
    if additional_info and additional_info != 'nan':
        sections.append(f"\\subsection*{{Additional Information}}\n{additional_info}")
    
    product_page_description = f"\\section*{{{plugin_name} Product Page Description}}\n\n" + "\n\n".join(sections)
    return product_page_description

# Function to generate an email announcement
def generate_email(row):
    plugin_name = escape_latex(row.get("Name of Plugin", ""))
    main_description = escape_latex(row.get("Main description (15 - 25 words)", ""))
    unique_features = escape_latex(row.get("What makes this plugin unique?", ""))
    pricing = escape_latex(row.get("Pricing", ""))
    
    email_content = f"""
    Subject: Announcing {plugin_name}

    Body:
    We are thrilled to introduce {plugin_name}, a groundbreaking audio plugin. {main_description}

    Unique Features:
    {unique_features}

    Pricing:
    {pricing}

    Call to Action:
    Visit our website to learn more and download {plugin_name}.
    """
    return email_content

# Function to generate a press release
def generate_press_release(row):
    plugin_name = escape_latex(row.get("Name of Plugin", ""))
    main_description = escape_latex(row.get("Main description (15 - 25 words)", ""))
    unique_features = escape_latex(row.get("What makes this plugin unique?", ""))
    pricing = escape_latex(row.get("Pricing", ""))
    
    press_release_content = f"""
    ## Press Release: {plugin_name}

    Headline: Introducing {plugin_name}

    Introduction:
    We are excited to announce the release of {plugin_name}. {main_description}

    Unique Features:
    {unique_features}

    Pricing:
    {pricing}

    Contact Information:
    For press inquiries, please contact press@example.com.
    """
    return press_release_content

# URL of the public Google Sheet in CSV format
csv_url = 'https://docs.google.com/spreadsheets/d/1WQyrrg9HlXh1SVBsMYRdR26NlrxgOilch1d4EVuHUXA/export?format=csv&gid=1387996182'

# Fetch the data from the Google Sheet
response = requests.get(csv_url)
if response.status_code != 200:
    raise Exception(f"Failed to fetch data: {response.status_code}, {response.text}")

# Load the data into a pandas DataFrame
df = pd.read_csv(csv_url)

# Process the first row in the dataframe to generate the product page description
product_page_description = generate_product_page_description(df.iloc[0])

# Generate email announcement
email_content = generate_email(df.iloc[0])

# Generate press release
press_release_content = generate_press_release(df.iloc[0])

# Save the contents to files
with open('product_page_description.tex', 'w') as f:
    f.write(product_page_description)

with open('email_announcement.txt', 'w') as f:
    f.write(email_content)

with open('press_release.txt', 'w') as f:
    f.write(press_release_content)

print("Files generated successfully.")

# Path to the main.tex file
tex_file_path = 'main.tex'

# Read the existing content of the main.tex file
with open(tex_file_path, 'r') as file:
    tex_content = file.read()

# Add the product page description to the main.tex file
updated_tex_content = tex_content.replace('\\subsection*{Installation}', '\\subsection*{Installation}\n' + product_page_description)

# Write the updated content back to the main.tex file
with open(tex_file_path, 'w') as file:
    file.write(updated_tex_content)

print("main.tex file updated successfully.")

# Commit and push the changes to the GitHub repository
repo = Repo('.')
commit_and_push_changes(repo, ['product_page_description.tex', 'email_announcement.txt', 'press_release.txt', tex_file_path])


Files generated successfully.
main.tex file updated successfully.
Changes pushed to GitHub repository successfully.


In the following section are some random functions for market research and keyword optimization

In [28]:
import requests
from bs4 import BeautifulSoup
from googlesearch import search

def fetch_keywords_from_google_autocomplete(keyword):
    url = f"https://www.google.com/complete/search?q={keyword}&cp=11&client=gws-wiz&xssi=t&hl=en&authuser=0&dpr=1.5&source=hp&dpr=1.5"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"HTTP error occurred: {response.status_code} for url: {url}")
        return []

    # Remove the ")]}'\n" prefix
    data = response.text.lstrip(")]}'\n")
    suggestions = eval(data)[0]

    cleaned_suggestions = []
    for suggestion in suggestions:
        # Clean HTML tags
        soup = BeautifulSoup(suggestion[0], 'html.parser')
        text = soup.get_text()
        # Replace HTML entities with spaces
        text = text.replace("<\\/b>", "")
        cleaned_suggestions.append(text.strip())
    
    return cleaned_suggestions

def google_search_urls(query, num_results=5):
    # Perform the search and get the URLs
    urls = search(query, num_results=num_results)
    return urls

# Example list of keywords to search
keywords_to_search = ["creative", "experimental"]
predefined_tags = ["Audio plugin"]

# Fetch keywords data
all_keywords_data = []
for tag in predefined_tags:
    for keyword in keywords_to_search:
        combined_keyword = f"{tag} {keyword}"
        keywords_data = fetch_keywords_from_google_autocomplete(combined_keyword)
        all_keywords_data.extend(keywords_data)

# Remove duplicates
all_keywords_data = list(set(all_keywords_data))

# Print the cleaned keywords
print("Cleaned Keywords:")
print(all_keywords_data)

# Fetch search results for each cleaned keyword
all_search_results = {}
for keyword in all_keywords_data:
    print(f"Fetching results for: {keyword}")
    search_results = google_search_urls(keyword)
    all_search_results[keyword] = search_results

# Print the search results
print("\nSearch Results:")
for keyword, results in all_search_results.items():
    print(f"\nKeyword: {keyword}")
    for result in results:
        print(result)


Cleaned Keywords:
['experimental plugins', 'creative plugins', 'creating audio plugins']
Fetching results for: experimental plugins
Fetching results for: creative plugins
Fetching results for: creating audio plugins

Search Results:

Keyword: experimental plugins
https://insider.dbsinstitute.ac.uk/8-free-experimental-plugins
https://www.kvraudio.com/forum/viewtopic.php?t=581459
https://www.reddit.com/r/VSTi/comments/161qyzg/what_are_the_best_free_plugins_for_ambient_and/
https://synthanatomy.com/2022/12/best-software-experimental-synthesizer-releases-2022.html
https://magneticmag.com/2024/03/top-vsts-for-glitches/
https://modwiggler.com/forum/viewtopic.php?t=240641
https://plus.pointblankmusicschool.com/?p=79379

Keyword: creative plugins
https://plus.pointblankmusicschool.com/7-creative-plugins-to-spark-inspiration-when-youre-stuck/
https://www.musicradar.com/news/top-new-effect-plugins-for-2024
https://www.databroth.com/blog/10-interesting-plugins-of-2022
https://www.elektronauts.com